* This file is for cleaning and analyzing extracted data into monthly summaries 
* Then exporting to a master csv 

In [7]:
import numpy as np
import pandas as pd
import csv
import os
import datetime

In [ ]:
for filename in sorted(os.listdir("./raw_data")):
    df = pd.read_csv(f"./raw_data/{filename}")
    #replace placeholder values with zero 
    df["MIN"] = df["MIN"].replace(9999.9, 0)
    df["MAX"] = df["MAX"].replace(9999.9, 0)
    grouped = df.copy().groupby("MONTH").agg({'PRCP' : ['mean', 'median', 'min', 'max'],
                                    'TEMP' : ['mean', 'median'],
                                    'MIN' : ['min'],
                                    'MAX' : ["max"],
                                    })

    #calculate variance
    var = df[["MONTH", "TEMP", "PRCP"]].groupby("MONTH").var()
    grouped["TEMP_var"] = var["TEMP"]
    grouped["PRCP_var"] = var["PRCP"]

    #flatten 
    grouped.columns = [f'{col[0]}_{col[1]}' for col in grouped.columns]
    grouped = grouped.reset_index()

    #reorder and rename columns 
    grouped["YEAR"] = int(filename[:4])
    grouped = grouped.rename({"MIN_min" : "TEMP_min", "MAX_max" : "TEMP_max", "TEMP_var_" : "TEMP_var", "PRCP_var_" : "PRCP_var"}, axis = 'columns')
    grouped = grouped[["MONTH", "YEAR", "TEMP_mean", "TEMP_median", "TEMP_min", "TEMP_max", "TEMP_var", "PRCP_mean", "PRCP_median", "PRCP_min", "PRCP_max", "PRCP_var"]]
    if int(filename[:4]) == 1950:
        grouped.to_csv('monthly_weather_stats.csv', mode='a', index=False, header=True)     #write to CSV with header if it's the first file
    else:
        grouped.to_csv('monthly_weather_stats.csv', mode='a', index=False, header=False)